<br>
<br>

# # Title: Audio Classifier :

<br>
<br>

- convert audio data to wavefrom 
- transform waveform to spectrogram
- classify capuchin bird calls


In [10]:
#
# 
# Downlaod data:  
#
#

# !pip install kagglehub

# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("kenjee/z-by-hp-unlocked-challenge-3-signal-processing")

# print("Path to dataset files:", path)

